# Importing and preparing rental apartments data

## Libraries and settings

In [1]:
# Libraries
import os
import re
import time
import fnmatch
import numpy as np
import pandas as pd

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Get current working directory
print(os.getcwd())

/workspaces/data_analytics/Week_14


## Importing data

In [19]:
# Show .csv - files in the directory
flist = fnmatch.filter(os.listdir('.'), '*.CSV')
for i in flist:
    print(i)

# Read the data to a pandas data frame
df = pd.read_csv('Gemeindestand_erweitert.CSV', sep=';', encoding='ISO-8859-1')

# Show first records of data frame
df.head()

Gemeindestand_erweitert.CSV
Velomarkt.CSV


,Ortschaftsname,PLZ,Zusatzziffer,Gemeindename,BFS-Nr,Kanton,E,N,Sprache,Validity,Bezirks-nummer,Bezirksname,Einwohner,Einwohnerdichte,Fahrraddiebstahl_pro1000Einwohner
0,Aeugst am Albis,8914,0,Aeugst am Albis,1,ZH,8.487911,47.268707,de,01.07.2008,101,Affoltern,57801,511.4,1.51
1,Aeugstertal,8914,2,Aeugst am Albis,1,ZH,8.493639,47.282708,de,01.07.2008,101,Affoltern,57801,511.4,1.51
2,Zwillikon,8909,0,Affoltern am Albis,2,ZH,8.433816,47.289572,de,01.07.2008,101,Affoltern,57801,511.4,1.51
3,Affoltern am Albis,8910,0,Affoltern am Albis,2,ZH,8.454392,47.278791,de,01.07.2008,101,Affoltern,57801,511.4,1.51
4,Bonstetten,8906,0,Bonstetten,3,ZH,8.462510,47.316036,de,01.07.2008,101,Affoltern,57801,511.4,1.51


## Count number of rows and columns in the data frame

In [23]:
# Dimension (rows, columns)
print('Dimension:', df.shape)

# Number of rows
print('Number of rows:', df.shape[0])

# Number of columns h
print('Number of columns:', df.shape[1])

Dimension: (5764, 15)
Number of rows: 5764
Number of columns: 15


## Get data types (raw-format from web scraping)

In [22]:
# Get data types (note that in pandas, a string is referred to as 'object')
df.dtypes

Ortschaftsname                        object
PLZ                                    int64
Zusatzziffer                           int64
Gemeindename                          object
BFS-Nr                                 int64
Kanton                                object
E                                    float64
N                                    float64
Sprache                               object
Validity                              object
Bezirks-nummer                        object
Bezirksname                           object
Einwohner                             object
Einwohnerdichte                       object
Fahrraddiebstahl_pro1000Einwohner     object
dtype: object

## Extract and save relevant information from raw data using regular expressions (regex)

### Extract number of rooms

In [5]:
# Extract values from 'rooms_area_price_raw' strings
rooms = []
for i in df['rooms_area_price_raw']:
    d1 = re.findall('(.*)Zimmer', i)
    try:
        d2 = d1[0].strip().replace(',', '.')
    except:
        d2 = None
    rooms.append(d2)

# Save as new variable in the pandas data frame
df['rooms'] = pd.Series(rooms, dtype="float64")
    
# Print first 5 values
print(df['rooms_area_price_raw'].head(5), '\n')
print(df['rooms'].head(5))

0      3 Zimmer, 49 m², CHF 1441.—
1    3,5 Zimmer, 65 m², CHF 1850.—
2                19 m², CHF 2686.—
3      2 Zimmer, 54 m², CHF 4853.—
4      2 Zimmer, 49 m², CHF 4335.—
Name: rooms_area_price_raw, dtype: object 

0    3.0
1    3.5
2    NaN
3    2.0
4    2.0
Name: rooms, dtype: float64


### Extract living area

In [6]:
# Extract values from 'rooms_area_price_raw' strings
area = []
for i in df['rooms_area_price_raw']:
    d1 = re.findall('Zimmer, (.*)m²', i)
    try:
        d2 = d1[0].strip()
    except:
        d2 = None
    area.append(d2)

# Save as new variable in the pandas data frame
df['area'] = pd.Series(area, dtype="Int64")

# Print first 5 values
print(df['rooms_area_price_raw'].head(5), '\n')
print(df['area'].head(5))

0      3 Zimmer, 49 m², CHF 1441.—
1    3,5 Zimmer, 65 m², CHF 1850.—
2                19 m², CHF 2686.—
3      2 Zimmer, 54 m², CHF 4853.—
4      2 Zimmer, 49 m², CHF 4335.—
Name: rooms_area_price_raw, dtype: object 

0      49
1      65
2    <NA>
3      54
4      49
Name: area, dtype: Int64


### Extract rental price

In [7]:
# Extract values from 'price_raw' strings
price = []
for i in df['price_raw']:
    d1 = re.findall('[0-9]+', i)
    try:
        d2 = d1[0].strip()
    except:
        d2 = None
    price.append(d2)

# Save as new variable in the pandas data frame
df['price'] = pd.Series(price, dtype="Int64")

# Print first 5 values
print(df['price_raw'].head(5), '\n')
print(df['price'].head(5))

0    CHF 1441.—
1    CHF 1850.—
2    CHF 2686.—
3    CHF 4853.—
4    CHF 4335.—
Name: price_raw, dtype: object 

0    1441
1    1850
2    2686
3    4853
4    4335
Name: price, dtype: Int64


### Get data types of all variables including the new ones

In [8]:
df.dtypes

web-scraper-order         object
web-scraper-start-url     object
rooms_area_price_raw      object
address_raw               object
price_raw                 object
description_raw           object
text_raw                  object
rooms                    float64
area                       Int64
price                      Int64
dtype: object

## Count and identify missing values (if any)

In [9]:
# Count missing values
print(pd.isna(df).sum())

# Identify rows with missing values
df[df.isna().any(axis=1)].head()

web-scraper-order          0
web-scraper-start-url      0
rooms_area_price_raw       0
address_raw                0
price_raw                  0
description_raw            0
text_raw                   0
rooms                     45
area                     135
price                     15
dtype: int64


,web-scraper-order,web-scraper-start-url,rooms_area_price_raw,address_raw,price_raw,description_raw,text_raw,rooms,area,price
2,1693998201-3,https://www.immoscout24.ch/de/immobilien/miete...,"19 m², CHF 2686.—","Cramerstrasse 8-12, 8004 Zürich, ZH",CHF 2686.—,«Studio Apartment Junior Balcony»,"19 m², CHF 2686.—Cramerstrasse 8-12, 8004 Züri...",NaN,<NA>,2686
5,1693998201-6,https://www.immoscout24.ch/de/immobilien/miete...,"34 m², CHF 3205.—","Binzmühlestr. 50, 8050 Zürich, ZH",CHF 3205.—,«Studio Apartment Senior Terrace»,"34 m², CHF 3205.—Binzmühlestr. 50, 8050 Zürich...",NaN,<NA>,3205
6,1693998201-7,https://www.immoscout24.ch/de/immobilien/miete...,"25 m², CHF 3308.—","Rotachstrasse 33, 8003 Zürich, ZH",CHF 3308.—,«1 Bedroom Apartment Junior Terrace»,"25 m², CHF 3308.—Rotachstrasse 33, 8003 Zürich...",NaN,<NA>,3308
7,1693998201-8,https://www.immoscout24.ch/de/immobilien/miete...,"43 m², CHF 4241.—","Militärstrasse 24, 8004 Zürich, ZH",CHF 4241.—,«2 Bedroom Apartment Junior»,"43 m², CHF 4241.—Militärstrasse 24, 8004 Züric...",NaN,<NA>,4241
8,1693998201-9,https://www.immoscout24.ch/de/immobilien/miete...,"19 m², CHF 2582.—","Cramerstrasse 8-12, 8004 Zürich, ZH",CHF 2582.—,«Studio Apartment Junior»,"19 m², CHF 2582.—Cramerstrasse 8-12, 8004 Züri...",NaN,<NA>,2582


## Count and identify duplicated values (if any)

In [10]:
# Count duplicated values
print(df.duplicated().sum())

# Identify rows with duplicated values, e.g.:
df[df[['web-scraper-order', 'price_raw', 'address_raw']].duplicated()]

0


,web-scraper-order,web-scraper-start-url,rooms_area_price_raw,address_raw,price_raw,description_raw,text_raw,rooms,area,price


### Save data to file

In [11]:
df.to_csv('apartments_data_prepared.csv', 
          sep=",", 
          encoding='utf-8',
          index=False)

### Jupyter notebook --footer info-- (please always provide this at the end of each submitted notebook)

In [12]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Linux | 6.8.0-1014-azure
Datetime: 2024-09-24 16:55:53
Python Version: 3.11.10
-----------------------------------
